In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import concatenate
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import glob
import cv2
import os
import locale
import os

In [7]:
file = "epsilon-3"

### Load in `f` data

In [8]:
df = pd.read_csv(file + "-f.csv", sep=",")
df.T

,0,1,2,3,4,5,6,7,8,9,...,12637,12638,12639,12640,12641,12642,12643,12644,12645,12646
current_player,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,...,1.000000,0.000000,1.000000,0.000000,1.0,0.0,1.000000,0.000000,1.0,0.000000
is_current_player_hand_winner,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,...,0.000000,1.000000,0.000000,1.000000,0.0,1.0,0.000000,1.000000,0.0,1.000000
is_current_player_game_winner,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,...,0.000000,1.000000,0.000000,1.000000,0.0,1.0,0.000000,1.000000,0.0,1.000000
current_player_end_hand_score_advantage,-20.000000,20.000000,-20.000000,20.000000,-20.000000,20.000000,13.000000,-13.000000,13.000000,-13.000000,...,-37.000000,37.000000,-37.000000,37.000000,-37.0,37.0,-37.000000,37.000000,-37.0,37.000000
current_player_score,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.000000,0.000000,20.000000,0.000000,...,6.000000,90.000000,6.000000,90.000000,6.0,90.0,6.000000,90.000000,6.0,90.000000
opponent_score,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.000000,0.000000,20.000000,...,90.000000,6.000000,90.000000,6.000000,90.0,6.0,90.000000,6.000000,90.0,6.000000
current_player_deadwood,39.000000,35.000000,39.000000,12.000000,29.000000,9.000000,55.000000,65.000000,52.000000,35.000000,...,53.000000,34.000000,49.000000,34.000000,49.0,29.0,49.000000,23.000000,49.0,7.000000
current_player_num_hit_cards,2.000000,11.000000,2.000000,9.000000,2.000000,5.000000,4.000000,8.000000,4.000000,8.000000,...,9.000000,5.000000,5.000000,5.000000,5.0,5.0,5.000000,4.000000,5.0,3.000000
num_melds,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,1.000000,0.000000,1.000000,0.0,1.0,0.000000,1.000000,0.0,2.000000
point_sum_melds,15.000000,30.000000,15.000000,51.000000,22.000000,51.000000,0.000000,0.000000,0.000000,29.000000,...,0.000000,35.000000,0.000000,35.000000,0.0,35.0,0.000000,35.000000,0.0,47.000000


In [80]:
f = [
#      'current_player',
#      'is_current_player_hand_winner',
#      'is_current_player_game_winner',
     'current_player_end_hand_score_advantage',
#      'current_player_score',
#      'opponent_score',
#      'current_player_deadwood',
     'current_player_num_hit_cards',
     'num_melds',
#      'point_sum_melds',
     'num_combos',
#      'point_sum_combos',
#      'num_knock_cache',
#      'point_sum_knock_cache',
#      'num_load_cards',
#      'point_sum_load_cards',
     'turns_taken',
#      'num_nearby_opponent_cards',
#      'discard_danger'
]


x = f[1:]
y = "current_player_end_hand_score_advantage"

print("x =", x)
print("y =", y)

x = ['current_player_num_hit_cards', 'num_melds', 'num_combos', 'turns_taken']
y = current_player_end_hand_score_advantage


In [30]:
features = df[f]

### Load in `m` data

In [13]:
card_matrices = []

with open(file + '-m.csv') as f:
    for line in f.readlines():
        card_matrices.append(eval(line))

card_matrices = np.array(card_matrices)

### split train and test data

In [53]:
trainFeaturesX, testFeaturesX, trainMatricesX, testMatricesX = train_test_split(df[x], card_matrices, test_size=0.25, random_state=42)
trainY, testY = train_test_split(df[[y]], test_size=0.25, random_state=42)

display(trainFeaturesX.T)
display(trainY.T)

,5579,7608,1437,1233,185,10501,9626,2465,3532,3449,...,4426,466,6265,5734,11284,11964,5191,5390,860,7270
current_player_num_hit_cards,7.0,3.0,9.0,5.0,3.0,1.0,7.0,1.0,5.0,7.0,...,5.0,4.0,4.0,7.0,11.0,7.0,6.0,6.0,4.0,3.0
num_melds,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,...,2.0,2.0,2.0,1.0,0.0,0.0,1.0,2.0,1.0,1.0
num_combos,3.0,0.0,4.0,2.0,1.0,0.0,5.0,0.0,1.0,4.0,...,2.0,1.0,1.0,3.0,5.0,4.0,3.0,2.0,2.0,1.0
turns_taken,2.0,6.0,18.0,16.0,7.0,12.0,13.0,17.0,6.0,6.0,...,13.0,19.0,21.0,12.0,3.0,8.0,9.0,7.0,8.0,2.0


,5579,7608,1437,1233,185,10501,9626,2465,3532,3449,...,4426,466,6265,5734,11284,11964,5191,5390,860,7270
current_player_end_hand_score_advantage,28.0,49.0,-1.0,4.0,-3.0,-9.0,-5.0,9.0,-13.0,-4.0,...,-16.0,-9.0,5.0,-6.0,7.0,19.0,-13.0,6.0,-15.0,-46.0


### create the MLP model

In [54]:
mlp = Sequential()
mlp.add(Dense(8, input_dim=trainFeaturesX.shape[1], activation="relu"))
mlp.add(Dense(4, activation="relu"))
if False:
    mlp.add(Dense(1, activation = "linear"))

### Create CNN model

In [55]:
height = len(card_matrices[0])
width = len(card_matrices[0][0])
depth = 1
filters = (16, 32, 64)

inputShape = (height, width, depth)
chanDim = -1

inputs = Input(shape=inputShape)

for (i, f) in enumerate(filters):
    # if this is the first CONV layer then set the input appropriately
    if i == 0:
        x = inputs

    # CONV => RELU => BN => POOL
    x = Conv2D(f, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

# flatten the volume, then FC => RELU => BN => DROPOUT
x = Flatten()(x)
x = Dense(16)(x)
x = Activation("relu")(x)
x = BatchNormalization(axis=chanDim)(x)
x = Dropout(0.5)(x)

# match the number of nodes coming out of the MLP
x = Dense(4)(x)
x = Activation("relu")(x)

if False:
    x = Dense(1, activation="linear")(x)

cnn = Model(inputs, x)

### Combine

In [56]:
combinedInput = concatenate([mlp.output, cnn.output])

x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)

model = Model(inputs=[mlp.input, cnn.input], outputs=x)

### Compile

In [57]:
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

### Train

In [58]:
model.fit(
    [trainFeaturesX, trainMatricesX], trainY,
    validation_data=([testFeaturesX, testMatricesX], testY),
    epochs=200, batch_size=8)

Epoch 1/200
1186/1186 [==============================] - 3s 2ms/step - loss: 100.6930 - val_loss: 99.9275
Epoch 2/200
1186/1186 [==============================] - 2s 2ms/step - loss: 99.8970 - val_loss: 99.9048
Epoch 3/200
1186/1186 [==============================] - 2s 2ms/step - loss: 99.8980 - val_loss: 100.1087
Epoch 4/200
1186/1186 [==============================] - 2s 2ms/step - loss: 99.8220 - val_loss: 99.8519
Epoch 5/200
1186/1186 [==============================] - 2s 2ms/step - loss: 99.7748 - val_loss: 99.7191
Epoch 6/200
1186/1186 [==============================] - 2s 2ms/step - loss: 99.6730 - val_loss: 99.3296
Epoch 7/200
1186/1186 [==============================] - 2s 2ms/step - loss: 99.4430 - val_loss: 99.1485
Epoch 8/200
1186/1186 [==============================] - 3s 2ms/step - loss: 99.2254 - val_loss: 99.3507
Epoch 9/200
1186/1186 [==============================] - 3s 2ms/step - loss: 98.2136 - val_loss: 99.5025
Epoch 10/200
1186/1186 [=============================

### Testing

In [82]:
preds = model.predict([testFeaturesX, testMatricesX])
diff = preds.T[0] - testY.values.T[0]

print("average difference =", np.average(diff))

average difference = -0.02577715799221881


In [78]:
print(model.predict([testFeaturesX, testMatricesX]).T[0][:5])

[ 8.046639  11.167872  -6.8544    -2.8229232 -7.781098 ]


In [79]:
print(testY.values.T[0][:5])

[  7.  10. -17.  -5.  -9.]


In [ ]:
model.save('best_model.h5')